In [11]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from model.rnn import *
from model.lstm import *
from model.metrics import *

In [ ]:
# Load dataset
data = pd.read_csv('../simplified_data.csv', index_col='Date', parse_dates=True)

#stock_column = 'BNP.PA_Close'
#data = data[[stock_column]].dropna()
features = data.columns.tolist()
data = data[features].values 

scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

In [13]:
look_back = 1
X, y = create_dataset(data, look_back)

# Split into training and testing sets
split = int(0.6 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

split = int(0.5 * len(X_test))
X_test, X_validation = X_test[:split], X_test[split:]
y_test, y_validation = y_test[:split], y_test[split:]

# Reshape the data for LSTM
X_train = reshape_data(X_train, look_back)
X_test = reshape_data(X_test, look_back)
X_validation = reshape_data(X_validation, look_back)

# Build and train the LSTM model
model = build_lstm_model((X_train.shape[1], X_train.shape[2]))  # Input shape (timesteps, features)
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict and evaluate
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculate metrics
rmse = calculate_rmse(y_test_rescaled, y_pred_rescaled)
nse = calculate_nse(y_test_rescaled, y_pred_rescaled)

print(f"RMSE: {rmse:.4f}")
print(f"NSE: {nse:.4f}")


ValueError: cannot reshape array of size 40128 into shape (1824,1,1)